# Wstęp do Sztucznej Inteligencji - rok akademicki 2021/2022

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia.

# Temat: Optymalizacja globalna: Prosty algorytm genetyczny cz. II
Głownym celem zajęć poświęconych alorytmom genetycznym jest stworzenie od podstaw (implentacja) prostego algorytmu genetycznego i późniejsze wykorzystanie go do rozwiązania przykładowych zadań optymalizacji globalnej.

W tym notatniku będą Państwo mieli za zadanie zaimplementować trzy operatory genetyczne (trzy funkcje) wykonywane w głównej pętli algorytmu genetycznego. 

__Ważne:__ Wszystkie te funkcja wykonywane będą w każdej iteracji algorytmu należy zatem zadbać o to aby było ona zaimplementowana w sposób wydajny.

## Import biblioteki numpy
Wszystkie funkcje należy tworzyć z wykorzystaniem biblioteki `numpy`.

In [ ]:
import numpy as np

## Przykładowa funkcja celu
Zadanie optymalizacji w wielu przypadakach sproawadza się optymalizacji odpowiednio sformułowanej funkcj, tzw. funkcji celu. Poniżej przykładowa prosta funkcja umożliwiająca testowanie zaimplemetowanych funkcji.  

In [ ]:
# testowa funkcja celu
# x - jednowymiarowa tablica ndarray
def obj_func(x):
    return (x**2).sum()

Przykład wywołania.

Wektor `x` to tak zwany osobnik, czyli jedno z możliwych (dopuszczalnych) rozwiązań.  

In [ ]:
# przykład wywołania
x = np.array([1.2, 0.1, 3, 2.1])
print(obj_func(x))

14.86


## Selekcja: Metoda koła ruletki
Metoda ruletki (koła ruletki)– jedna z najbardziej podstawowych metod selekcji. Polega na utworzeniu koła ruletki z polami odpowiadającymi poszczególnym osobnikom. Wielkość pól jest proporcjonalna do wartości funkcji przystosowania (celu). Proces selekcji oparty jest na obrocie ruletką tyle razy ile osobników jest w populacji i na wyborze za każdym razem jednego osobnika do nowej populacji. Pewne osobniki są wybierane więcej niż jeden raz, niektóre dokładnie raz, a niektóre wcale.

Koło ruletki w praktyce:
- Koło zestępujemy odcinkiem $[0,1]$, który dzielimy na tyle podprzedziałów ile osobników w populacji.
- Rozmiar każdego podprzedziału jest proporcjonalny do wartość funkcji celu danego osobnika, tzn. im większa wartość funkcji celu tym odcinek powinnien być większy. Ponieważ cały odcinek ma długośc 1 to rozmiary podprzedziałów można utożsamoć z prawdopodobieństwe przejścia osobnika do nowej populacji i policzyć ze wzoru $p_i= \frac{e_i}{\sum_i{e_i}}$, gdzie $i$ to numer osobnika, a $e_i$ to wartość funkcji celu $i$-tego osobnika.
- Zamiast kręcić kołem, losujemy liczbę z przedziału $[0,1]$ i sprawdzamy w którym podprzedziale się ona zawiera i osobnika o numerze zgodnym z numerem tego podprzedziału wybieramy do nowej populacji.  

Warto zwrócić na fakt, że tak zaproponowany wzór na długości przedziałów $p_i$ zadziała tylko wtedy gdy wszystkie wartości funkcji celu będą dodatnie. Aby to zapewnić wystarczy na początku do wektora wartości funkcji celu dodać odpowiednią stałą zapewniającą dodatniość (lub przynajmniej nieujemność) wszystkich elementów.



### Zadanie 1:
Zaimplementuj operator selekcji. Selekcja metodą koła ruletki.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `evaluated_pop` - tablica `ndarray` ocen osobników.
- `new_pop` - nowa populacja wybranych osobników (zakodowanych), tablica `ndarray`.

In [ ]:
pop = np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
epop = np.array([0.30176899, 0.40998959, 0.88449532, 0.70135276, 1.75858481])

In [ ]:
def roulette(pop, evaluated_pop):

  new_evaluated_pop=[i/evaluated_pop.sum() for i in evaluated_pop]
  new_new_evaluated_pop = new_evaluated_pop.copy()

  for i in range(1,len(new_evaluated_pop)):
    new_new_evaluated_pop[i] = new_evaluated_pop[i]+new_new_evaluated_pop[i-1] 

  new_pop = np.array([[],[],[],[],[],[]])

  print(new_new_evaluated_pop)

  for i in range(len(pop)):
    zmienna = np.random.rand()
    print(zmienna)
    for j in range(len(pop)):
      if zmienna < new_new_evaluated_pop[j]:
        new_pop[i] = pop[j]
        break
  return new_pop 
  #0.2 0.3 0.1 0.4
  # 0.2 0.5 0.6 1

print(roulette(pop,epop))

[0.07439712652420721, 0.17547460105476728, 0.393535145420539, 0.5664443301045647, 1.0]
0.2890506393449592


ValueError: ignored

## Krzyżowanie
Operacja krzyżowania polega na wymianie fragmentów łańcuchów dwóch osobników rodzicielskich. Krzyżowanie jest kluczowym operatorem w algorytmach genetycznych, stanowiącym o ich sile i efektywności.

Ideą operatorów krzyżowania jest wymiana kodu genetycznego pomiędzy osobnikami, tak jak to się dzieje w naturze. Stworzono wiele teorii i rodzajów krzyżowań, które stosowane są do różnych rodzajów zadań i są zależne od sposobu kodowania. Dla potrzeb klasycznego algorytmu genetycznego opisano operator krzyżowania jednopunktowego. 

Krzyżowanie jednopunktowe – krzyżowanie zachodzi z pewnym prawdopodobieństwem `pk`; dla każdego osobnika losuje się liczbę i sprawdza, czy zachodzi krzyżowanie. Następnie dobiera się wybrane osobniki losowo w pary. Losuje się liczbę określającą miejsce krzyżowania i wymienia kod.

Podpowiedź: w praktyce krzyżowanie to można zaimplementować na dwa sposoby.

Sposób 1. Mając tablicę `pop` z parzystą liczbą osobników jako wiersze, dla każdych kolejnych dwóch wierszy wylosuj liczbę losową z przedziału `[0,1]`. Jeśli jest ona mniejsza niż prawdopodobieństwo krzyżowania, skrzyżuj osobniki i oboje potomków umieść w populacji wynikowej. Jeśli wartość losowa jest większa niż prawdopodobieństwo krzyżowania, skopiuj oboje rodziców do populacji wynikowej. 

Sposób 2. Mając tablicę `pop` z dowolną (niekoniecznie parzystą) liczbą osobników jako wiersze, dla każdego wiersza wylosuj liczbę losową z przedziału `[0,1]`. Jeśli jest ona mniejsza niż prawdopodobieństwo krzyżowania, skrzyżuj osobnika z osobnikiem w następnym wierszu i dowolnego z potomków (jednego!) umieść w populacji wynikowej. Jeśli wartość losowa jest większa niż prawdopodobieństwo krzyżowania, skopiuj aktualny wiersz do populacji wynikowej. Uwaga: ostatni wiersz krzyżowany jest z pierwszym (następnym w sensie "modulo").

![krzyzowanie.PNG](http://torus.uck.pk.edu.pl/~amarsz/images/krzyzowanie.PNG)




### Zadanie 2
Zaimplementuj operator krzyżowania. Krzyżowanie jedno-punktowe.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `pk` - prawdopodobieństwo krzyżowania dla pary osobników. Liczba rzeczywista z przedziału $[0,1]$.
- `new_pop` - nowa populacja osobników po krzyżowaniu (zakodowanych), tablica `ndarray`.

In [ ]:
def cross(pop, pk):
    ### TWÓJ KOD TUTAJ
    return new_pop

## Mutacja
Mutacja polega na wprowadzeniu do istniejących, zakodowanych osobników, pewnych losowych zmian. Mutacja tworzy nowego osobnika na bazie jednego i tylko jednego rodzica. Jest wiele metod tworzenia nowych osobników za pomocą operatora mutacji.

Podstawową formę mutacji można zapisać następująco:
$$x'= m(x)$$
gdzie: $x$ jest osobnikiem rodzica, $m$ funkcją mutacji, $x'$ to potomek, osobnik zmutowany.

Mutacja również zachodzi z pewnym zadanym prawdopodobieństwem `pm`. Można ją zaimplementować na dwa sposoby.

Sposób 1. Dla każdego osobnika losuje się liczbę sprawdzając czy będzie on podlegał mutacji. Jeśli tak to losuje się gen, który będzie zmutowany i dokonuje się mutacji (tu: negacja bitu). 

Sposób 2. Dla danego osobnika, dla każdego genu (bitu) osobno losuje się liczbę sprawdzającą czy będzie podlegał mutacji. Jeśli tak, dokonuje się jego mutacji (u nas: negacja bitu).

Mutacja w klasycznym algorytmie genetycznym odgrywa drugorzędną rolę. Częstość mutacji potrzebna do uzyskania dobrych wyników w empirycznych badaniach nad algorytmami genetycznymi jest rzędu jeden do tysiąca skopiowanych bitów. W naturalnych populacjach częstość jest równie mała lub nawet mniejsza. 

W przypadku osobników zakodowanych binarnie (klasyczny algorytm genetyczny) nie ma problemu ze stosowaniem mutacji (po prostu zamieniamy wartość genu na przeciwny). 

### Zadanie 3
Zaimplementuj operator mutacji. Mutacja bitowa.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `pm` prawdopodobieństwo mutacji dla pojedynczego bitu. Liczba rzeczywista z przedziału $[0,1]$.
- `new_pop` - nowa populacja osobników po mutacji (zakodowanych), tablica `ndarray`.

In [ ]:
def mutate(pop, pm):
    ### TWÓJ KOD TUTAJ
    return new_pop

&copy; Katedra Informatyki, Politechnika Krakowska